In [1]:
import os

import numpy as np
import pandas as pd
# Torch
import torch
import torchvision
from torchvision import datasets,transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import pathlib

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cuda


Load Dataset

In [5]:
transformer = transforms.Compose([transforms.Resize((150,150)),
                                  transforms.Grayscale(num_output_channels=1),
                                  transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(), # chnages from 0-255 to 0-1, numpy to tensors
                                  transforms.Normalize([0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5])
                                 ])

In [7]:
#DataLoader

batchSize=64
train_path = '/content/drive/MyDrive/Deep Learning Project/chest_xray/train'
train_loader = DataLoader(torchvision.datasets.ImageFolder(
                          train_path,
                          transform=transformer),
                          batch_size = batchSize,
                          shuffle=True)
val_path = '/content/drive/MyDrive/Deep Learning Project/chest_xray/val'
val_loader = DataLoader(torchvision.datasets.ImageFolder(
                          val_path,
                          transform=transformer),
                          batch_size = batchSize)

test_path = '/content/drive/MyDrive/Deep Learning Project/chest_xray/test'
test_loader = DataLoader(torchvision.datasets.ImageFolder(
                          test_path,
                          transform=transformer),
                          batch_size = batchSize,
                          shuffle=False)


In [ ]:
# from torch.utils.data import Subset, ConcatDataset
# from torch.utils.data import Dataset
# # Specify the number of samples you want to include from class 2
# num_samples_class2 = 1500

# # Get indices of samples belonging to class 1 and class 2
# class2_indices = [idx for idx, (img, label) in enumerate(train_loader.dataset) if label == 'PNEUMONIA']

# # Create a Subset dataset containing all samples from class 1
# class1_dataset = train_loader.dataset

# # Take a subset of samples from class 2 based on the specified number
# class2_subset_indices = class2_indices[:num_samples_class2]

# # Create a Subset dataset containing samples from class 2
# class2_subset_dataset = Subset(train_loader.dataset, class2_subset_indices)

# # Concatenate the Subset datasets to create a single dataset containing samples from both classes
# combined_dataset = ConcatDataset([class1_dataset, class2_subset_dataset])

# # Create DataLoader for the combined dataset
# combined_train_loader = DataLoader(combined_dataset, batch_size=64, shuffle=True)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from torch.utils.data import Subset

# # Define the number of images you want to load
# train_subset_indices = range(3000)
# test_subset_indices = range(500)

# # Create subsets of the dataset
# train_subset = Subset(train_loader.dataset, train_subset_indices)
# test_subset = Subset(test_loader.dataset, test_subset_indices)

# # Create loaders for the subsets
# train_loader_subset = DataLoader(train_subset, batch_size=batchSize, shuffle=True)
# test_loader_subset = DataLoader(test_subset, batch_size=batchSize, shuffle=False)


In [28]:

total_images_train = len(train_loader.dataset)

print("Total number of images in the training dataset:", total_images_train)

total_images_test = len(test_loader.dataset)

print("Total number of images in the test dataset:", total_images_test)

total_images_train = len(val_loader.dataset)

print("Total number of images in the validation dataset:", total_images_train)

print(total_images_train)


Total number of images in the training dataset: 2880
Total number of images in the test dataset: 397
Total number of images in the validation dataset: 227
2880


In [10]:
sample_image, sample_label = train_loader.dataset[0]
print(sample_image.shape)

torch.Size([1, 150, 150])


In [11]:
root=pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1]for j in root.iterdir()])
print(classes)

['NORMAL', 'PNEUMONIA']


Convolutional Neural Network Architetuire

In [12]:
class ConvNet(nn.Module):
  def __init__(self,num_classes=2):
    super(ConvNet,self).__init__()
    # input shape (256,1, 150,150)

    #output shape ((width-kernel+2P)/stride)+1 150-3+2+1=150
    self.conv1 =nn.Conv2d(in_channels=1,out_channels=20,kernel_size=5,stride=1,padding=2) # outptu channels is the number of kernels used
    #shape = 256*12*150*150
    self.bn1 = nn.BatchNorm2d(num_features=20)
    self.relu1 = nn.ReLU()
    self.pool=nn.MaxPool2d(kernel_size=2)
    #shape = 256*12*75*75 (reduce image size by 2)

    self.conv2 = nn.Conv2d(in_channels=20,out_channels = 32,kernel_size =5,stride=1,padding=2)
    #256*20*75*75
    self.bn2=nn.BatchNorm2d(num_features=32)
    self.dropout1 = nn.Dropout(0.5) # 50 % of neurons will be zero
    self.relu2 = nn.ReLU()
    #256*20*75*75
    self.conv3 = nn.Conv2d(in_channels=32,out_channels =64,kernel_size =3,stride =1 ,padding=1)
#     #256*32*75*75
    self.bn3 = nn.BatchNorm2d(num_features=64)
    self.relu3 =nn.ReLU()


    self.fc = nn.Linear(in_features=64*75*75,out_features = 2)

  def forward(self,input):
    output = self.conv1(input)
    output = self.bn1(output)
    output = self.relu1(output)
    output = self.pool(output)
    output = self.conv2(output)
    output = self.bn2(output)
    output= self.dropout1(output)
    output = self.relu2(output)
    output = self.conv3(output)
    output = self.bn3(output)
    output = self.relu3(output)

    output = output.view(-1,64*75*75)
    output = self.fc(output)
    return output






In [13]:
model = ConvNet(num_classes=2).to(device)

In [21]:
optimizer =Adam(model.parameters(),lr = 0.001,weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()
# loss_function = nn.BCELoss()

In [15]:
num_epochs =5

Model Training

In [30]:

from torch.autograd import Variable
best_accuracy=0.0
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3, threshold=0.01, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)
iterations=[]
for epoch in range(num_epochs):

    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0


    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        optimizer.zero_grad()

        outputs=model(images)
        #using cross entropy loss (can explore BCE also)
        loss=loss_function(outputs,labels)
        #to calculate the loss wrt teh parameters and start backpropagation
        loss.backward()
        #to update the weights
        optimizer.step()


        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        # train_accuracy+=int(torch.sum(prediction==labels.data))
        train_accuracy += torch.sum(prediction == labels.data).item()

    print(train_accuracy)
    train_accuracy=train_accuracy/len(train_loader.dataset)
    train_loss=train_loss/len(train_loader.dataset)
    # Evaluation on testing dataset
    model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/total_images_test


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    # scheduler.step(test_accuracy)

    #Save the best model
    # if test_accuracy>best_accuracy:
    #     torch.save(model.state_dict(),'best_checkpoint.model')
    #     best_accuracy=test_accuracy


2819.0
227
Epoch: 0 Train Loss: tensor(0.6051) Train Accuracy: 0.9788194444444445 Test Accuracy: 0.8035264483627204
2842.0
227
Epoch: 1 Train Loss: tensor(0.3517) Train Accuracy: 0.9868055555555556 Test Accuracy: 0.7783375314861462
2794.0
227
Epoch: 2 Train Loss: tensor(0.7973) Train Accuracy: 0.9701388888888889 Test Accuracy: 0.7707808564231738
2830.0
227
Epoch: 3 Train Loss: tensor(0.4554) Train Accuracy: 0.9826388888888888 Test Accuracy: 0.7657430730478589
2845.0
227
Epoch: 4 Train Loss: tensor(0.1776) Train Accuracy: 0.9878472222222222 Test Accuracy: 0.7984886649874056
2855.0
227
Epoch: 5 Train Loss: tensor(0.1519) Train Accuracy: 0.9913194444444444 Test Accuracy: 0.7858942065491183


KeyboardInterrupt: 

In [ ]:
from torch.autograd import Variable
import matplotlib.pyplot as plt

best_accuracy = 0.0
train_accuracies = []
test_accuracies = []

#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3, threshold=0.01, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)

for epoch in range(num_epochs):

    #Evaluation and training on training dataset
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

        optimizer.zero_grad()

        outputs = model(images)
        #using cross entropy loss (can explore BCE also)
        loss = loss_function(outputs, labels)
        #to calculate the loss wrt the parameters and start backpropagation
        loss.backward()
        #to update the weights
        optimizer.step()

        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data, 1)

        train_accuracy += torch.sum(prediction == labels.data).item()

    train_accuracy = train_accuracy / len(train_loader.dataset)
    train_loss = train_loss / len(train_loader.dataset)

    train_accuracies.append(train_accuracy)

    # Evaluation on testing dataset
    model.eval()
    test_accuracy = 0.0

    for i, (images, labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())

        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))

    test_accuracy = test_accuracy / total_images_test
    test_accuracies.append(test_accuracy)

    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) + ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    # scheduler.step(test_accuracy)

# Plotting
iterations = range(1, num_epochs + 1)
plt.plot(iterations, train_accuracies, label='Training Accuracy')
plt.plot(iterations, test_accuracies, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracy Over Epochs')
plt.legend()
plt.show()

# Save the best model
# if test_accuracy > best_accuracy:
#     torch.save(model.state_dict(), 'best_checkpoint.model')
#     best_accuracy = test_accuracy


Epoch: 0 Train Loss: tensor(0.1871) Train Accuracy: 0.9888888888888889 Test Accuracy: 0.8337531486146096
Epoch: 1 Train Loss: tensor(0.1064) Train Accuracy: 0.99375 Test Accuracy: 0.7884130982367759
Epoch: 2 Train Loss: tensor(0.1146) Train Accuracy: 0.9934027777777777 Test Accuracy: 0.7556675062972292
Epoch: 3 Train Loss: tensor(0.2395) Train Accuracy: 0.9888888888888889 Test Accuracy: 0.8085642317380353
Epoch: 4 Train Loss: tensor(0.1978) Train Accuracy: 0.9902777777777778 Test Accuracy: 0.8136020151133502
Epoch: 5 Train Loss: tensor(0.3380) Train Accuracy: 0.9850694444444444 Test Accuracy: 0.801007556675063
Epoch: 6 Train Loss: tensor(0.1373) Train Accuracy: 0.9916666666666667 Test Accuracy: 0.7959697732997482
Epoch: 7 Train Loss: tensor(0.0818) Train Accuracy: 0.9961805555555555 Test Accuracy: 0.7959697732997482


In [ ]:

from torch.autograd import Variable
best_accuracy=0.0
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3, threshold=0.01, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)

for epoch in range(num_epochs):

    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        optimizer.zero_grad()

        outputs=model(images)
        #using cross entropy loss (can explore BCE also)
        loss=loss_function(outputs,labels)
        #to calculate the loss wrt teh parameters and start backpropagation
        loss.backward()
        #to update the weights
        optimizer.step()


        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))


    print(train_accuracy)
    print(total_images_train)
    train_accuracy=train_accuracy/total_images_train
    train_loss=train_loss/total_images_train
    # Evaluation on testing dataset
    model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/total_images_test


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    # scheduler.step(test_accuracy)

    #Save the best model
    # if test_accuracy>best_accuracy:
    #     torch.save(model.state_dict(),'best_checkpoint.model')
    #     best_accuracy=test_accuracy


Accuracy: 0.79

In [ ]:
class ConvNet(nn.Module):
  def __init__(self,num_classes=2):
    super(ConvNet,self).__init__()
    # input shape (256,1, 150,150)

    #output shape ((width-kernel+2P)/stride)+1 150-3+2+1=150
    self.conv1 =nn.Conv2d(in_channels=1,out_channels=12,kernel_size=3,stride=1,padding=1) # outptu channels is the number of kernels used
    #shape = 256*12*150*150
    self.bn1 = nn.BatchNorm2d(num_features=12)
    self.relu1 = nn.ReLU()
    self.pool=nn.MaxPool2d(kernel_size=2)
    #shape = 256*12*75*75 (reduce image size by 2)

    self.conv2 = nn.Conv2d(in_channels=12,out_channels = 20,kernel_size =3,stride=1,padding=1)
    #256*20*75*75
    self.relu2 = nn.ReLU()
    #256*20*75*75

    self.conv3 = nn.Conv2d(in_channels=20,out_channels =32,kernel_size =3,stride =1 ,padding=1)
    #256*32*75*75
    self.bn2 = nn.BatchNorm2d(num_features=32)
    self.relu3 =nn.ReLU()

    self.fc = nn.Linear(in_features=32*75*75,out_features = 2)

  def forward(self,input):
    output = self.conv1(input)
    output = self.bn1(output)
    output = self.relu1(output)
    output = self.pool(output)
    output = self.conv2(output)
    output = self.relu2(output)
    output = self.conv3(output)
    output = self.bn2(output)
    output = self.relu3(output)
    output = output.view(-1,32*75*75)
    output = self.fc(output)
    return output






Accuracy: 0.78

In [ ]:
class ConvNet(nn.Module):
  def __init__(self,num_classes=2):
    super(ConvNet,self).__init__()
    # input shape (256,1, 150,150)

    #output shape ((width-kernel+2P)/stride)+1 150-3+2+1=150
    self.conv1 =nn.Conv2d(in_channels=1,out_channels=12,kernel_size=3,stride=1,padding=1) # outptu channels is the number of kernels used
    #shape = 256*12*150*150
    self.bn1 = nn.BatchNorm2d(num_features=12)
    self.relu1 = nn.ReLU()
    self.pool=nn.MaxPool2d(kernel_size=2)
    #shape = 256*12*75*75 (reduce image size by 2)

    self.conv2 = nn.Conv2d(in_channels=12,out_channels = 20,kernel_size =3,stride=1,padding=1)
    #256*20*75*75
    self.bn2=nn.BatchNorm2d(num_features=20)
    self.relu2 = nn.ReLU()
    #256*20*75*75
    self.conv3 = nn.Conv2d(in_channels=20,out_channels =64,kernel_size =3,stride =1 ,padding=1)
#     #256*32*75*75
    self.bn3 = nn.BatchNorm2d(num_features=64)
    self.relu3 =nn.ReLU()


    self.fc = nn.Linear(in_features=64*75*75,out_features = 2)

  def forward(self,input):
    output = self.conv1(input)
    output = self.bn1(output)
    output = self.relu1(output)
    output = self.pool(output)
    output = self.conv2(output)
    output = self.bn2(output)
    output = self.relu2(output)
    output = self.conv3(output)
    output = self.bn3(output)
    output = self.relu3(output)

    output = output.view(-1,64*75*75)
    output = self.fc(output)
    return output




